## RQ1 Creating FAO forest map

[Add Description]

Need to create a FAO Definition Approximation following the steps from Johnson et al (2023)

Steps:
1. Data Preparation
    1. Clip the 5 GER LULC shapefiles to the Natura 2000 areas
    2. Merge the clipped 5 GER LULC shapefiles into 1 shapefile
    3. Vectorise the clipped 5m JAXA raster
2. Intersect JAXA and GER LULC shapefiles
3. Conditional Reclassing
4. Convert to 5m raster

In [9]:
# SETUP

# Note: this .ipynb file depends on files & folder structures created in rq1_step1_data_prep.ipynb

# Import packages
import glob
import os

import pandas as pd
import geopandas as gpd


### Step 1: Data Preparation

#### Step 1.1: Clip GER LULC SHPs

In the rq1_step1_data_prep.ipynb file, I clipped all the output **rasters** to the Germany Natura 2000 areas - in this first data prep step, I do the same for the GER LULC shapefiles, as I will be working with vector data for creating the FAO-aligned forest map. 


In [ ]:
# CLIP GER LULC SHPS

# Store paths to reprojected GER LULC SHPs in a list
ger_lulc_paths = glob.glob('./processing/clc5_class*xx_3035_DE.shp')

# Create a function which which clips the shp to the Germany Natura areas (& saves to processing folder)
def clip_shp_to_natura(input_paths):
    # Load Germany Natura 2000 areas
    natura_de_gdf = gpd.read_file("./outputs/natura2000_3035_DE.shp")
    # Iterate through the GER LULC shp paths 
    for path in input_paths:
        # Open the shp for each path 
        ger_lulc_shp = gpd.read_file(path)
        
        # Clip input GER LULC shp to Natura shp
        shp_clip  = gpd.clip(ger_lulc_shp, natura_de_gdf)

        # For output file naming: extract the input file name (with extension)
        name_w_ext = os.path.split(path)[1] 
        # For output file naming: remove extension from input file name 
        name_wo_ext = os.path.splitext(name_w_ext)[0]
        # For output file naming: create the new name for clipped shp
        new_name = name_wo_ext + "_clipped.shp"

        # Write the reprojected shp to the processing folder
        shp_clip.to_file('./processing/' + new_name)

# Run the function for the German LULC zipped shps
clip_shp_to_natura(ger_lulc_paths)


#### Step 1.2: Merge clipped GER LULC to 1 shp 

In order to make the next steps easier, all the clipped GER LULC shps can be merged into 1 master shp. 

Help for merging/appending shps: https://geopandas.org/en/stable/docs/user_guide/mergingdata.html 

In [ ]:
# MERGE GER LULC SHPS

# Load all the GER LULC SHPs
ger_lulc_class1_shp = gpd.read_file("./outputs/clc5_class1xx_3035_DE_clipped.shp")
ger_lulc_class2_shp = gpd.read_file("./outputs/clc5_class2xx_3035_DE_clipped.shp")
ger_lulc_class3_shp = gpd.read_file("./outputs/clc5_class3xx_3035_DE_clipped.shp")
ger_lulc_class4_shp = gpd.read_file("./outputs/clc5_class4xx_3035_DE_clipped.shp")
ger_lulc_class5_shp = gpd.read_file("./outputs/clc5_class5xx_3035_DE_clipped.shp")

# Append the shapefiles together
merged_ger_lulc_shp = pd.concat([ger_lulc_class1_shp,
                                 ger_lulc_class2_shp,
                                 ger_lulc_class3_shp,
                                 ger_lulc_class4_shp,
                                 ger_lulc_class5_shp
                                 ])

#### Step 1.3: Reclassify & vectorise clipped Jaxa

In order to implement the workflow for creating the FAO-aligned forest map, all input data must be vectors. In this step I vectorise the clipped JAXA 5m raster - but first I reclassify the raster to get rid of unneeded information.

Help with reclassifying (use gdal_calc): https://gis.stackexchange.com/questions/245170/reclassifying-raster-using-gdal 

Help with raster to vector: https://py.geocompx.org/05-raster-vector

